In [7]:
import sys
import tensorflow as tf
import numpy as np
import json, os
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

from sklearn.metrics import precision_recall_curve, average_precision_score
from tensorflow.keras.metrics import AUC, Precision, Recall

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2

# Set your batch size and number of epochs
batch_size = 16
nb_epochs = 20

#Win 10
train_path = 'MERGED_ALL_CLEAN_balanced_ready/train'
val_path = 'MERGED_ALL_CLEAN_balanced_ready/val'
test_path = 'MERGED_ALL_CLEAN_balanced_ready/test'

# Initialize ImageDataGenerator without any augmentation
datagen = ImageDataGenerator(rescale=1./255)

# Create the train, validation, and test data generators
train_data_gen = datagen.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'  ######### Changed to 'binary' for 2 classes #########
)

valid_data_gen = datagen.flow_from_directory(
    directory=val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'  ######### Changed to 'binary' for 2 classes #########
)

test_data_gen = datagen.flow_from_directory(
    directory=test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',  ######### Changed to 'binary' for 2 classes #########
    shuffle=False
)

# Load the VGG19 model without the top layers
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract weights for the desired layers
vgg_weights = {
    'block1_conv1': vgg19_model.get_layer('block1_conv1').get_weights(),
    'block1_conv2': vgg19_model.get_layer('block1_conv2').get_weights(),
    'block2_conv1': vgg19_model.get_layer('block2_conv1').get_weights(),
    'block2_conv2': vgg19_model.get_layer('block2_conv2').get_weights()
}

def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = Conv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(1, activation='sigmoid', name='fc3')(x) ######### Changed to 1 unit with 'sigmoid' activation for 2 classes #########
    
    model = Model(inputs=input_img, outputs=x)
    return model

custom_model =  build_model()
#custom_model.summary()

VGG_conv1_weights = vgg_weights['block1_conv1']
VGG_conv2_weights = vgg_weights['block1_conv2']
VGG_conv4_weights = vgg_weights['block2_conv1']
VGG_conv5_weights = vgg_weights['block2_conv2']

custom_model.get_layer('Conv1_1').set_weights(VGG_conv1_weights)
custom_model.get_layer('Conv1_2').set_weights(VGG_conv2_weights)
custom_model.get_layer('Conv2_1').set_weights(VGG_conv4_weights)
custom_model.get_layer('Conv2_2').set_weights(VGG_conv5_weights)

######### Freeze the transferred layers #########
custom_model.get_layer('Conv1_1').trainable = False
custom_model.get_layer('Conv1_2').trainable = False
custom_model.get_layer('Conv2_1').trainable = False
custom_model.get_layer('Conv2_2').trainable = False
######### End of freezing #########

#custom_model.summary()  

# Define optimizer, early stopping, and checkpoint
opt = Adam(learning_rate=0.0001)
es = EarlyStopping(patience=5, restore_best_weights=True)
chkpt = ModelCheckpoint(filepath='best_model_todate.weights.h5', save_best_only=True, save_weights_only=True)

######### Add additional metrics #########


custom_model.compile(
    loss='binary_crossentropy',  ######### Changed to 'binary_crossentropy' for 2 classes #########
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')], 
    optimizer=opt
)

######### End of metrics #########
######### End of metrics #########
'''
# Train the model
history = custom_model.fit(
    train_data_gen,
    validation_data=valid_data_gen,
    epochs=nb_epochs,
    callbacks=[es, chkpt]
)

history_dict = history.history
with open('training_history.json', 'w') as f:
    json.dump(history_dict, f)
'''


# Load pre-trained weights
custom_model.load_weights('best_model_todate.weights_aug_2000.h5')



# Evaluate the model on the test data
test_loss, test_acc, test_auc, test_precision, test_recall = custom_model.evaluate(test_data_gen)

print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")



with open('training_history_2k.json', 'r') as f:
    history_dict = json.load(f)


# Choose ticks every few epochs
tick_positions = np.arange(0, len(history_dict['loss']), step=1)  # Adjust the step as needed

# Plotting
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history_dict['loss'], label='Training Loss')
plt.plot(history_dict['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Set x-axis ticks with the chosen positions
plt.xticks(ticks=tick_positions)

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(history_dict['accuracy'], label='Training Accuracy')
plt.plot(history_dict['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Set x-axis ticks with the chosen positions
plt.xticks(ticks=tick_positions)

plt.savefig('Training_Validation_Plot.png', bbox_inches='tight')




# Generate predictions
y_pred = custom_model.predict(test_data_gen)
y_pred = (y_pred > 0.5).astype(int)

# Compute confusion matrix
cm = confusion_matrix(test_data_gen.classes, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.savefig('Confusion_Matrix.png', bbox_inches='tight')




y_pred_prob = custom_model.predict(test_data_gen).ravel()

# Compute ROC curve and AUC
fpr, tpr, _ = roc_curve(test_data_gen.classes, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('ROC_Curve.png', bbox_inches='tight')

# https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc#:~:text=The%20area%20under%20the%20ROC,curve%20(AUC)%20of%201.0.





precision, recall, _ = precision_recall_curve(test_data_gen.classes, y_pred_prob)
average_precision = average_precision_score(test_data_gen.classes, y_pred_prob)

# Plot Precision-Recall curve
plt.figure()
plt.step(recall, precision, where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
plt.savefig('Precision-Recall_Curve.png', bbox_inches='tight')




############### MISSCLASIFIED PICS

def plot_misclassified_images(model, data_gen, num_images=5):
    # Get predictions and true labels
    y_pred_prob = model.predict(data_gen, verbose=1)
    y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions
    y_true = data_gen.classes
    
    # Generate indices
    batch_size = data_gen.batch_size
    steps = len(data_gen)
    misclassified_indices = []

    for i in range(steps):
        # Get a batch of data
        images, labels = next(data_gen)
        predictions = model.predict(images, verbose=0).flatten()
        batch_indices = np.where((predictions > 0.5).astype(int) != labels)[0]
        
        if len(batch_indices) > 0:
            for idx in batch_indices:
                misclassified_indices.append((i * batch_size) + idx)

        if len(misclassified_indices) >= num_images:
            break

    # Plot a few misclassified images
    plt.figure(figsize=(12, 12))
    for i, idx in enumerate(misclassified_indices[:num_images]):
        batch_idx = idx // batch_size
        img_idx = idx % batch_size

        # Get image and label
        data_gen.reset()
        for j in range(batch_idx + 1):
            images, labels = next(data_gen)
        
        img = images[img_idx]
        true_label = labels[img_idx]
        pred_label = (model.predict(np.expand_dims(img, axis=0)) > 0.5).astype(int)[0][0]
        
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f"True: {true_label}\nPred: {pred_label}")
        plt.axis('off')
    plt.savefig('Missclassified_images.png', bbox_inches='tight')

# Usage
plot_misclassified_images(custom_model, test_data_gen)

Found 19535 images belonging to 2 classes.
Found 4188 images belonging to 2 classes.
Found 4188 images belonging to 2 classes.


ValueError: Layer count mismatch when loading weights from file. Model expected 11 layers, found 0 saved layers.

In [5]:
with open("training_history_2k.json", 'r') as file:
    data = json.load(file)

# Print the content of the JSON file
print(json.dumps(data, indent=4))  # Pretty print with indentation

{
    "accuracy": [
        0.7502337098121643,
        0.8058978319168091,
        0.8228945136070251,
        0.8328800797462463,
        0.8585450649261475,
        0.8731622099876404,
        0.8887141942977905,
        0.8987422585487366,
        0.9097476005554199,
        0.9179484844207764,
        0.9263618588447571,
        0.9360074996948242,
        0.9411489963531494,
        0.9481601119041443,
        0.9550862312316895,
        0.9602702260017395,
        0.9614599943161011,
        0.9654542207717896,
        0.9679187536239624,
        0.9737825989723206
    ],
    "auc": [
        0.8005639910697937,
        0.862351655960083,
        0.8853951096534729,
        0.9037367701530457,
        0.9264711737632751,
        0.9393771290779114,
        0.9495150446891785,
        0.9577470421791077,
        0.9664487242698669,
        0.9709383249282837,
        0.9754785895347595,
        0.9808979630470276,
        0.9840002059936523,
        0.986899197101593,
        0.9